In [1]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm

In [2]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 108.0.5359
[WDM] - Get LATEST chromedriver version for 108.0.5359 google-chrome
[WDM] - Driver [C:\Users\krish\.wdm\drivers\chromedriver\win32\108.0.5359.71\chromedriver.exe] found in cache


In [3]:
driver.get('https://www.imdb.com/title/tt3498820/reviews?ref_=tt_urv')

count = 0

while (count<11):
    try:
        time.sleep(1)
        driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'load-more-trigger'))))
        count +=1
    except count==11:
        break

In [4]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel = Selector(text = first_review.get_attribute('innerHTML'))

In [5]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []

error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel.css('.text.show-more__control::text').extract()
            review = ' '.join(review)
        except:
            review = np.NaN
        try:
            review_date = sel.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel.css('a.title::text').extract_first()
        except:
            review_title = np.NaN

        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)

    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'review_date':review_date_list,
    'author':author_list,
    'rating':rating_list,
    'review_title':review_title_list,
    'review':review_list
    })

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:05<00:00, 50.50it/s]


In [6]:
review_df.tail()

,review_date,author,rating,review_title,review
295,17 May 2016,cdennis39,2,Marvelously Blah....\n,Action for actions sake. Plot holes galore and...
296,28 April 2016,dimbost,9,Perfect example of how to make a great super ...,I'm not usually keen on giving such high score...
297,13 April 2016,ernesto-43482,10,An Amazing Movie\n,Russo Brothers did an amazing job with this fi...
298,27 April 2016,morphion2,9,The Great Responsibility\n,Eight years into Marvel Studios' multi-propert...
299,25 December 2016,mitrapourmand,5,Magic ruined it for me\n,"Winter soldier was the best marvel movie , it ..."


In [7]:
review_df.head()

,review_date,author,rating,review_title,review
0,24 July 2021,snoozejonc,8,Huge amount of spectacle & character moments\n,When an incident involving Wanda Maximoff caus...
1,30 May 2021,Venomage03,10,Captain America: Civil War is one of the best...,This movie has a fantastic plot and brings all...
2,7 April 2021,andreascaloni,9,"GRIPPING, BOLD AND SMART\n","Captain America Civil War is a gripping, bold,..."
3,30 July 2021,AlsExGal,9,A master class in screenwriting\n,What impresses me about this movie is how much...
4,31 March 2021,JWick23,None,I could do this all day...\n,Captain America: Civil War was a great sequel ...


In [8]:
review_df.to_csv('civilwar.csv', index=False)